# Problem 21
### (Discretizing eigenfunction problems into eigenvalue ones. Solving spatial part of wave eqn w/ Dirichlet, Neumann, Periodic BCs)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.sparse.linalg as splin
from time import time

Will discretize $\frac{d^2X}{dx^2}=\lambda X$

### Part C) Calculate the first 8 normal modes and their eigenvalues for Dirichlet BCs, end points x=0,L.

Let us work in a normalized dimension, BCs: X(0)=0, X(1)=0.

In [2]:
# Will use the following function in Part C,D,E to generate matrix representations of d2X/dx2
def deriv_matrix(dim, bc='Dirichlet'):
    # dim x dim matrix for discretizing
    d2X_dx2 = np.diag(np.ones(dim-1),k=1)+np.diag(np.ones(dim-1),k=-1)+np.diag(-2*np.ones(dim),k=0)
    if bc=='Neumann':
        d2X_dx2[0,0] = -1
        d2X_dx2[dim-1,dim-1] = -1
    elif bc=='Neumann-Nonsym':
        d2X_dx2[0,1] = 2
        d2X_dx2[dim-1,dim-2] = 2
    elif bc=='Periodic':
        d2X_dx2[0,dim-1] = 1
        d2X_dx2[dim-1,0] = 1
    else:
        pass
    return d2X_dx2

*Will plot the modes in MATLAB (for the rest of the problem as well), using [V,D]=eigs(A,8,'sm') is much easier than sorting the entries of eigenvectors here in Python. See the solutions PDF for those.*

Let us see how accurate the eigenvalues are for some dim values. Note that we factored out the $h^2$ factor of the matrix and need to rescale the eigenvalues by it.

In [3]:
def numerical_checker(sorted_eigs,analytical_fn,ctr=[0,8],is_verbose=False):
    # Let us compare the eigenvalues with the analytical solutions:
    ave_err_pc = 0
    for i in range(ctr[0],ctr[1]):
        lambda_eff = sorted_eigs[i]
        actual = analytical_fn(i)
        error_pc = 100*(lambda_eff - actual)/actual
        if is_verbose is True:
            print(f"Eigval of Mode {i+1}: {lambda_eff}\t Error: {error_pc}%")
        ave_err_pc += error_pc
    ave_err_pc /= (ctr[1]-ctr[0])
    print(f"Average of above: {ave_err_pc}%")


dim = 72  # Toggle this
x_array = np.linspace(0,1,dim+2)
h_sqr = (x_array[1]-x_array[0])**2
dirichlet_eig = lambda n: -1*((n+1)*np.pi)**2

A = deriv_matrix(dim,bc='Dirichlet')
sorted_eigs = sorted(np.real(sp.linalg.eig(A)[0])/h_sqr, reverse=True)
print(f"Dim: {dim}; The ground is Mode 1")        

numerical_checker(sorted_eigs,dirichlet_eig, is_verbose=True)

Dim: 72; The ground is Mode 1
Eigval of Mode 1: -9.868081240433431	 Error: -0.015432844053591289%
Eigval of Mode 2: -39.45405154694796	 Error: -0.061719944435624695%
Eigval of Mode 3: -88.70312451336521	 Error: -0.1388270170241113%
Eigval of Mode 4: -157.52410219390453	 Error: -0.2466969594813521%
Eigval of Mode 5: -245.78954398079543	 Error: -0.38524990782147484%
Eigval of Mode 6: -353.33600259494943	 Error: -0.5543833156322004%
Eigval of Mode 7: -479.964326752684	 Error: -0.7539720557556993%
Eigval of Mode 8: -625.4400299480758	 Error: -0.9838685443152709%
Average of above: -0.3925188235649156%


The eigenvalues found for dim=8 are as follows: [-9.769795432682805, -37.90080021472556, -81.0, -133.86899521795723, -190.1310047820425, -242.9999999999997, -286.0991997852739, -314.23020456731655]

For dim=8, we get an error of -1.0% for the eigval of the first mode and -50% for the eigval of the eighth mode. We need a 72x72 dimensional matrix representation to have at most 1% absolute error in terms of eigenvalues. The error (in absolute) is higher for eigenvalues of larger absolute value compared to smaller ones for any dim value, the absolute error of any mode decreases as we take dim larger.

Now, let us compare the speed of various eigenvalue finders that could suit our matrix representation.
We could try eigsh and eigs of scipy.sparse, eigvals and eigvalsh of numpy

In [4]:
def err_print_range(ctr,dim,bc='Dirichlet',func=dirichlet_eig,is_verbose=False,is_complete=False):
    # Unfortunately, the functions require some hands-on tweaking to be compatible with numerical_checker, hence the repetitiveness
    x_array = np.linspace(0,1,dim+2)
    h_sqr = (x_array[1]-x_array[0])**2
    A = deriv_matrix(dim,bc)
    A_sparse = sp.sparse.dia_matrix(A)
    
    print("\nNumpy eigvalsh-----")
    time2 = time()
    eigvals = np.linalg.eigvalsh(A)
    time2 = time()-time2
    print(f"Time elapsed: {time2}")
    sorted_eigs = sorted(np.real(eigvals)/h_sqr, reverse=True)
    numerical_checker(sorted_eigs,func,ctr,is_verbose)
    
    if is_complete is True:
        print("\nNumpy eigvals-----")
        time2 = time()
        eigvals = np.linalg.eigvals(A)
        time2 = time()-time2
        print(f"Time elapsed: {time2}")
        sorted_eigs = sorted(np.real(eigvals)/h_sqr, reverse=True)
        numerical_checker(sorted_eigs,func,ctr,is_verbose)
    
        print("\nScipy Sparse eigsh-----")
        time2 = time()
        eigvals = splin.eigsh(A_sparse, k=dim-1,return_eigenvectors=False)  # Cannot use k=dim, biggest eigval is thus unavailable
        time2 = time()-time2
        print(f"Time elapsed: {time2}")
        sorted_eigs = [0]+sorted(np.real(eigvals)/h_sqr, reverse=True)  # Need to fix the off-by-one error
        numerical_checker(sorted_eigs,func,ctr,is_verbose)
    
        print("\nScipy Sparse eigs-----")
        time2 = time()
        eigvals = splin.eigs(A_sparse, k=dim-2,return_eigenvectors=False)  # Cannot use k=dim nor dim-1
        time2 = time()-time2
        print(f"Time elapsed: {time2}")
        sorted_eigs = [0,0]+sorted(np.real(eigvals)/h_sqr, reverse=True)  # Need to fix the off-by-two error
        numerical_checker(sorted_eigs,func,ctr,is_verbose)
    else:
        pass

err_print_range([2,8],800,is_complete=True)


Numpy eigvalsh-----
Time elapsed: 0.015633821487426758
Average of above: -0.004251532102156799%

Numpy eigvals-----
Time elapsed: 0.8870153427124023
Average of above: -0.004251532535967718%

Scipy Sparse eigsh-----
Time elapsed: 0.6017584800720215
Average of above: -0.004251532340128643%

Scipy Sparse eigs-----
Time elapsed: 1.4004955291748047
Average of above: -0.004251532419138482%


We can see that Numpy's eigvalsh is both the most accurate and by far the fastest of the four methods, it was necessary to go up to dim=800 just to see a non-zero time value for eigvalsh

### Part D) Calculate the first 9 normal modes and their eigenvalues for Neumann BCs.

We will compare the two matrix representations for Neumann BCs, one is symmetric and the other is non-sym.

In [5]:
neumann_eig = lambda n: -1*(n*np.pi)**2
print("Ground mode is Mode 1")

print("Symmetric:")
err_print_range([1,9],80,bc='Neumann',func=neumann_eig,is_verbose=True)
print("\nNon-Symmetric:")
err_print_range([1,9],80,bc='Neumann-Nonsym',func=neumann_eig,is_verbose=True)

Ground mode is Mode 1
Symmetric:

Numpy eigvalsh-----
Time elapsed: 0.0
Eigval of Mode 2: -10.116586449227615	 Error: 2.5024513455777044%
Eigval of Mode 3: -40.45074675389067	 Error: 2.4629385080163817%
Eigval of Mode 4: -90.95570783745596	 Error: 2.397110856863295%
Eigval of Mode 5: -161.55359471059998	 Error: 2.305008985953186%
Eigval of Mode 6: -252.13555054880786	 Error: 2.1866896796708004%
Eigval of Mode 7: -362.5619045416855	 Error: 2.042225866066268%
Eigval of Mode 8: -492.6623872552393	 Error: 1.871706556654352%
Eigval of Mode 9: -642.2363931749946	 Error: 1.6752367729316895%
Average of above: 2.18042107146671%

Non-Symmetric:

Numpy eigvalsh-----
Time elapsed: 0.0
Eigval of Mode 2: -10.641083786161337	 Error: 7.81672044511558%
Eigval of Mode 3: -42.53663987301814	 Error: 7.746567502551458%
Eigval of Mode 4: -95.60471474834098	 Error: 7.63092066766639%
Eigval of Mode 5: -169.71226349200754	 Error: 7.471546347690706%
Eigval of Mode 6: -264.6797419531299	 Error: 7.270658963358597

Clearly, the symmetric matrix representation yields better results for a given dim value, and the eigenvalue of the ground mode is garbage for the non-symmetric one whereas the symmetric matrix yields 4.66e-12.

### Part E) Calculate and plot the first 8 normal modes and their eigenvalues for periodic BCs

The first mode corresponds to constant rotation, and has eigval 0. Numerically, the first 8 distinct eigenvalues are as follows:

In [6]:
dim = 80
x_array = np.linspace(0,1,dim)
h_sqr = (x_array[1]-x_array[0])**2

A = deriv_matrix(dim,bc='Periodic')
sorted_eigs = sorted(np.real(np.linalg.eigvalsh(A))/h_sqr, reverse=True)
print(f"Eigval of Mode 1:\t {sorted_eigs[0]}")
for i in range(1,15,2):
    print(f"Eigval of Mode {i+1},{i+2}:\t {sorted_eigs[i]}")

Eigval of Mode 1:	 1.5630628302106082e-12
Eigval of Mode 2,3:	 -38.47784034309278
Eigval of Mode 4,5:	 -153.6741326914891
Eigval of Mode 6,7:	 -344.8786535961928
Eigval of Mode 8,9:	 -610.912563603889
Eigval of Mode 10,11:	 -950.1356751941204
Eigval of Mode 12,13:	 -1360.4565650807924
Eigval of Mode 14,15:	 -1839.3454685322204


*Reminder: See the solutions pdf for the plots of modes*